In [1]:
# Run below code to set up the environment
import os
repo_path = 'Handwriting-Transformers'
if not os.path.exists(repo_path) and os.getcwd().split('/')[-1] != repo_path:
    !git clone https://github.com/ankanbhunia/Handwriting-Transformers
if os.getcwd().split('/')[-1] != repo_path:
    %cd Handwriting-Transformers
if not os.path.exists('files'): # Get the model and data files
    %pip install --upgrade --no-cache-dir gdown
    !gdown --id 16g9zgysQnWk7-353_tMig92KsZsrcM6k
    !unzip files.zip && rm files.zip

/home/taha_adeel/Desktop/Sem-7/Deep Learning/Project/Handwriting-Transformers


In [2]:
# Import the libraries
import os
from data.dataset import TextDataset, TextDatasetval
import torch
import cv2
import numpy as np
from models.model import TRGAN
from params import *
from torch import nn
from data.dataset import get_transform
import pickle
from PIL import Image
import tqdm
import shutil

In [3]:
data_path = 'files/IAM-32.pickle'
model_path = 'files/iam_model.pth'
num_examples = 15

In [4]:
# Load the model and the dataset
print ('(1) Loading dataset files...')
TextDatasetObjval = TextDatasetval(base_path = data_path, num_examples = num_examples)
datasetval = torch.utils.data.DataLoader(
            TextDatasetObjval,
            batch_size=8,
            shuffle=True,
            num_workers=0,
            pin_memory=True, drop_last=True,
            collate_fn=TextDatasetObjval.collate_fn)

print ('(2) Loading model...')
model = TRGAN()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda:0')
model.netG.load_state_dict(torch.load(model_path, map_location=device))
print (model_path+' : Model loaded Successfully')

(1) Loading dataset files...
(2) Loading model...


/home/taha_adeel/anaconda3/envs/tensorflow/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/taha_adeel/anaconda3/envs/tensorflow/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initialize network with N02
initialize network with N02
initialize network with N02


/home/taha_adeel/anaconda3/envs/tensorflow/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/taha_adeel/anaconda3/envs/tensorflow/lib/python3.11/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


files/iam_model.pth : Model loaded Successfully


In [5]:
# Save the images for fid calculations

# real_path, fake_path = model.save_images_for_fid_calculation(datasetval, epoch=None, mode='test')

real_data = 'train'
real_path = '../demo_output/real_fid'
fake_path = '../demo_output/fake_fid'
if os.path.isdir(real_path): shutil.rmtree(real_path)
if os.path.isdir(fake_path): shutil.rmtree(fake_path)

os.makedirs(real_path)
os.makedirs(fake_path)

# Generate the fake images
for step,data in enumerate(datasetval): 
    ST = data['simg'].to(device)
    fakes = model.netG.Eval(ST, model.eval_text_encode) 
    fake_images = torch.cat(fakes, 1).detach().cpu().numpy()

    for i in range(fake_images.shape[0]):
        for j in range(fake_images.shape[1]):
            cv2.imwrite(os.path.join(fake_path, str(step*batch_size + i)+'_'+str(j)+'.png'), 255*(fake_images[i,j]))

# Load real data
if real_data == 'train':
    TextDatasetObj = TextDataset(num_examples = model.eval_text_encode.shape[1])
    dataset_real = torch.utils.data.DataLoader(
                TextDatasetObj,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0,
                pin_memory=True, drop_last=True,
                collate_fn=TextDatasetObj.collate_fn)
elif real_data == 'test':
    TextDatasetObjval = TextDatasetval(num_examples = model.eval_text_encode.shape[1])
    dataset_real = torch.utils.data.DataLoader(
                TextDatasetObjval,
                batch_size=batch_size,
                shuffle=True,
                num_workers=0,
                pin_memory=True, drop_last=True,
                collate_fn=TextDatasetObjval.collate_fn) 
    
# Save the real images
for step,data in enumerate(dataset_real): 
    real_images = data['simg'].numpy()
    for i in range(real_images.shape[0]):
        for j in range(real_images.shape[1]):
            cv2.imwrite(os.path.join(real_path, str(step*batch_size + i)+'_'+str(j)+'.png'), 255*(real_images[i,j])) 


In [6]:
# Compute the FID score using the pytorch-fid library
import pytorch_fid.fid_score as fid

fid_score = fid.calculate_fid_given_paths([real_path, fake_path], device='cuda', dims=2048, batch_size=1, num_workers=8)
print ('FID Score : ' + str(fid_score))

100%|██████████| 10240/10240 [02:14<00:00, 76.25it/s]


FID Score : 16.660481699993966
